# Introduction

Amazon Personalize is a machine learning service that makes it easy for developers to add individualized 
recommendations to customers who use their applications. It reflects the vast experience that Amazon has 
in building personalization systems.

You can use Amazon Personalize in a variety of scenarios, such as giving users recommendations based on 
their preferences and behavior, personalized re-ranking of search results, and personalizing content for 
emails and notifications. 

In this lab, we will walk you through 

In particular, we will see how to:
1. [Initialize Amazon Personalize client](#Initialize-Amazon-Personalize-Client)
2. [Prepare and import data](#Prepare-and-Import-Data)
3. [Create a solution](#Create-a-solution)
4. [Evaluate a solution](#Evaluate-a-solution)
5. [Create a capmpaign](#Create-a-campaign)
6. [Get recommendations](#Get-Recommendations)


## [Initialize Amazon Personalize Client]()

### Import relevant liabraries

In [36]:
import boto3
import json
import numpy as np
import pandas as pd
import time

### Install the service JSON files for Amazon Personalize.

In [37]:
!wget -N https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize.json
!wget -N https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize-runtime.json
!wget -N https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize-events.json
!aws configure add-model --service-model file://`pwd`/personalize.json --service-name personalize
!aws configure add-model --service-model file://`pwd`/personalize-runtime.json --service-name personalize-runtime

--2019-04-26 09:05:55--  https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize.json
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.233.144
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.233.144|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘personalize.json’ not modified on server. Omitting download.

--2019-04-26 09:05:56--  https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize-runtime.json
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.233.144
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.233.144|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘personalize-runtime.json’ not modified on server. Omitting download.

--2019-04-26 09:05:56--  https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize-events.json
Resolving s3-us-west-2.amazonaws.com

### Initialize boto3 client for Amazon Personalize

In [38]:
personalize = boto3.client(service_name='personalize', endpoint_url='https://personalize.us-east-1.amazonaws.com', region_name='us-east-1')
personalize_runtime = boto3.client(service_name='personalize-runtime', endpoint_url='https://personalize-runtime.us-east-1.amazonaws.com', region_name='us-east-1')

## [Prepare and Import Data]()

### DataSet and Schemas

Before you add a dataset to Amazon Personalize, you're required to define a schema for the dataset.

Amazon Personalize recognizes three types of historical datasets. Each type has an associated schema with a name key whose value matches the dataset type. The three types are:

1. Users – This dataset is intended to provide metadata about your users. This includes information such as age, gender, and loyalty membership, among others, which can be important signals in personalization systems.

2. Items – This dataset is intended to provide metadata about your items. This includes information such as price, SKU type, and availability, among others.

3. Interactions – This dataset is intended to provide historical interaction data between users and items.

At the minimum you need a Interactions Schema. In case you want to take advantage of User attributes like GENDER, AGE then you need to explicitly define a Users Schema. Similarly if you want to take advantage of Item attributes then you need to explicitly define Items Schema. For example in case of Item like movies you would like model to learn on features like GENRE etc.

In this example, we will define the minimum required Interactions schema.

### Create Interactions Schema

In [39]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "veb-DEMO-schema-1",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateSchema operation: Another resource with Arn arn:aws:personalize:us-east-1:976421102596:schema/veb-DEMO-schema-1 already exists.

### Download and Explore the Dataset

In [4]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

--2019-04-17 07:48:47--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ml-100k.zip’ not modified on server. Omitting download.

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflat

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3,881250949
1,186,302,3,891717742
...,...,...,...,...
99998,13,225,2,882399156
99999,12,203,3,879959583


In [ ]:
bucket = "veb-personalize-demo"           # replace with the name of your S3 bucket
filename = "DEMO-movie-lens-100k.csv"

### Formating your input Data 

In [5]:
data = data[data['RATING'] > 3.6]                # keep only movies rated 3.6 and above
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)


### Uploading to Amazon S3 Bucket

You have to give Amazon Personalize permission to access the Amazon S3 bucket. We will create a policy and attach it to the bucket. 

In [ ]:
bucket = "veb-personalize-demo"           # replace with the name of your S3 bucket
filename = "DEMO-movie-lens-100k.csv"
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### Importing your Data

Import your training data into Amazon Personalize by first creating an empty dataset group and then an empty dataset in that dataset group. Next, create an import job that populates the dataset with data from your Amazon S3 bucket. For more information about datasets, see Datasets and Schemas.

### Create an Empty Dataset Group

In [9]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-dataset-group-1"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:976421102596:dataset-group/DEMO-dataset-group-1",
  "ResponseMetadata": {
    "RequestId": "6592d61c-02f7-4751-87e9-617eea11aeb9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 17 Apr 2019 07:56:38 GMT",
      "x-amzn-requestid": "6592d61c-02f7-4751-87e9-617eea11aeb9",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for Dataset Group to Have ACTIVE Status

In [10]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: ACTIVE


### Create Dataset

In [11]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = 'veb-dataset-group-1'
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:976421102596:dataset/DEMO-dataset-group-1/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "dfb8b940-6ae0-400d-bcaa-cfdbf1d36497",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 17 Apr 2019 07:58:44 GMT",
      "x-amzn-requestid": "dfb8b940-6ae0-400d-bcaa-cfdbf1d36497",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Prepare, Create, and Wait for Dataset Import Job

Import job, which is run on behalf of Amazon Personalize service, will need to access data 
from S3. By default access to S3 buckets are blocked. Let's create a bucket policy and also 
create a readAccessOnly roles for Amazon Personalize Service. We need to create a bucket 
policyan IAM role, which has permission to access S3. 
This role will be assumed by Import job. 

### Create a Policy and attach it to the bucket

In [12]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

#### Create S3 Read Only Access Role

In [14]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-group-1"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
);

iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
);

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::976421102596:role/PersonalizeS3Role-group-1


#### Create Dataset Import Job

In [15]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-dataset-import-job-1",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:976421102596:dataset-import-job/DEMO-dataset-import-job-1",
  "ResponseMetadata": {
    "RequestId": "dfa420ff-1dd7-4d24-b37b-d73d7431111a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 17 Apr 2019 08:00:54 GMT",
      "x-amzn-requestid": "dfa420ff-1dd7-4d24-b37b-d73d7431111a",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Dataset Import Job and Dataset Import Job Run to Have ACTIVE Status

In [17]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE


## [Create a solution]()


A solution is the term Amazon Personalize uses for a trained machine learning model that 
makes recommendations to customers. Creating a solution entails optimizing the model 
to deliver the best results for a specific business need. Amazon Personalize uses 
"recipes" to create these personalized solutions.

A recipe in Amazon Personalize is made up of an algorithm with hyperparameters, and a 
feature transformation.

### Select Recipe

In [ ]:
List all the available receipes in Amazon Personalize

In [18]:
list_recipes_response = personalize.list_recipes()

list_recipes_response

{'recipes': [{'name': 'aws-deepfm',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-deepfm',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal())},
  {'name': 'aws-ffnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-ffnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal())},
  {'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo

### Create and Wait for Solution

#### Create Solution

In [ ]:
Create solution 

Lets choose the aws-hrnn policy for this demo. You can read details on all other receipes here.
https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn" # aws-hrnn selected for demo purposes

In [19]:
create_solution_response = personalize.create_solution(
    name = "DEMO-dataset-group-solution-1",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:976421102596:solution/DEMO-dataset-group-solution-1",
  "ResponseMetadata": {
    "RequestId": "83a7b7b7-ec98-4e79-a0a2-9495dcd095e0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 17 Apr 2019 08:20:26 GMT",
      "x-amzn-requestid": "83a7b7b7-ec98-4e79-a0a2-9495dcd095e0",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Solution to Have ACTIVE Status

In [24]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_response = personalize.describe_solution(
        solutionArn = solution_arn
    )
    status = describe_solution_response["solution"]["status"]
    print("Solution: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

print(solution_arn)

Solution: ACTIVE
arn:aws:personalize:us-east-1:976421102596:solution/DEMO-dataset-group-solution-1


### Create Solution Version

In [25]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:976421102596:solution/DEMO-dataset-group-solution-1/f4503e8f",
  "ResponseMetadata": {
    "RequestId": "3133aea7-646d-4467-b578-04f045ceeb19",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 17 Apr 2019 08:52:52 GMT",
      "x-amzn-requestid": "3133aea7-646d-4467-b578-04f045ceeb19",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Wait for Solution Version to Have ACTIVE Status

In [28]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("Solution: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

print(solution_version_arn)

Solution: ACTIVE
arn:aws:personalize:us-east-1:976421102596:solution/DEMO-dataset-group-solution-1/f4503e8f


## [Evaluate a solution]()

In [29]:
get_solution_metrics_response = personalize.get_solution_metrics(solutionVersionArn = solution_version_arn)
print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:976421102596:solution/DEMO-dataset-group-solution-1/f4503e8f",
  "metrics": {
    "_num_evaluation_users": 96.0,
    "_num_unique_items": 1448.0,
    "_user_history_length_10_pct_quantile": 11.0,
    "_user_history_length_50_pct_quantile": 41.0,
    "_user_history_length_90_pct_quantile": 136.0,
    "_user_history_length_mean": 57.84375,
    "coverage": 0.281767955801105,
    "mean_reciprocal_rank": 0.05529997169888137,
    "normalized_discounted_cumulative_gain_at_10": 0.07896038136003791,
    "normalized_discounted_cumulative_gain_at_25": 0.10501386106099224,
    "normalized_discounted_cumulative_gain_at_5": 0.07211416070616289,
    "precision_at_10": 0.011458333333333334,
    "precision_at_25": 0.008749999999999999,
    "precision_at_5": 0.018750000000000003
  },
  "ResponseMetadata": {
    "RequestId": "19bfcc95-c2ac-4b96-be6f-db410bfc15f5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-

## [Create a campaign]()

#### Create campaign

In [30]:
create_campaign_response = personalize.create_campaign(
    minProvisionedTPS =1,
    name = "DEMO-dataset-group-1-campaign",
    solutionVersionArn = solution_version_arn
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:976421102596:campaign/DEMO-dataset-group-1-campaign",
  "ResponseMetadata": {
    "RequestId": "1384b06d-400c-403f-84b7-e5f5cbc3f756",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 17 Apr 2019 09:38:08 GMT",
      "x-amzn-requestid": "1384b06d-400c-403f-84b7-e5f5cbc3f756",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Campaign to Have ACTIVE Status

In [31]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


## [Get recommendations]()

#### Select a User and an Item

In [33]:
items = pd.read_csv('./ml-100k/u.item', encoding='latin-1', sep='|', usecols=[0,1], header=None)
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

items

USER: 115
ITEM: Mother Night (1996)


,ITEM_ID,TITLE
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
...,...,...
1680,1681,You So Crazy (1994)
1681,1682,Scream of Stone (Schrei aus Stein) (1991)


#### Call GetRecommendations

In [34]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
    itemId = str(item_id)
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

Recommendations: [
  "Clockwork Orange, A (1971)",
  "Aliens (1986)",
  "E.T. the Extra-Terrestrial (1982)",
  "Seven (Se7en) (1995)",
  "Some Like It Hot (1959)",
  "Empire Strikes Back, The (1980)",
  "GoodFellas (1990)",
  "Rear Window (1954)",
  "Usual Suspects, The (1995)",
  "Jaws (1975)",
  "African Queen, The (1951)",
  "Shining, The (1980)",
  "Psycho (1960)",
  "Clerks (1994)",
  "Star Trek: The Wrath of Khan (1982)",
  "Wrong Trousers, The (1993)",
  "Bridge on the River Kwai, The (1957)",
  "True Romance (1993)",
  "Silence of the Lambs, The (1991)",
  "Highlander (1986)",
  "One Flew Over the Cuckoo's Nest (1975)",
  "Batman (1989)",
  "Raiders of the Lost Ark (1981)",
  "Right Stuff, The (1983)",
  "Get Shorty (1995)"
]
